In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /home/aman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [10]:
train.head()

,post.id,user.id,gender,topic,sign,date,text,age
0,1,11869,male,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",15
1,2,11869,male,Student,Leo,"13,May,2004",These are the team members: Drewe...,15
2,3,11869,male,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,15
3,4,11869,male,Student,Leo,"12,May,2004",testing!!! testing!!!,15
4,5,16332,male,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,33


In [11]:
# Function to clean reviews and return filtered sentences
def clean_review(raw_review):
    review_text = str(raw_review)
    # Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    # Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    # Getting list of stop words
    stops = set(stopwords.words("english"))                  
    # Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    # Join the words back into one string separated by space and return
    return " ".join(meaningful_words)

In [21]:
# Initializing the Vectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 10000, \
                             ngram_range=(1,2))

In [22]:
# Cleaning Train Blog text
num_blogs = train["text"].size
clean_train_blogs = []

for i in range( 0, num_blogs ):
    clean_train_blogs.append(clean_review(train["text"][i]))

# Getting vectors for train data
train_data_features = vectorizer.fit_transform(clean_train_blogs)

MemoryError: 

In [43]:
# Cleaning Test Blog text
num_blogs = test["text"].size
clean_test_blogs = []

for i in range( 0, num_blogs ):
    clean_test_blogs.append(clean_review(test["text"][i]))

# Getting vectors for train data
test_data_features = vectorizer.fit_transform(clean_test_blogs)

In [44]:
# Initializing and fitting logistic regression model
from sklearn.linear_model import LinearRegression
linearRegression = LinearRegression()
lr = linearRegression.fit(train_data_features, train["age"])

In [50]:
preds = lr.predict(test_data_features)

In [51]:
solution = pd.DataFrame({"user.id":test["user.id"], "age":preds})

In [52]:
solution

,user.id,age
0,4876,27.923501
1,4876,32.266310
2,12227,23.959428
3,12227,19.754001
4,12227,23.994093
5,12227,25.865604
6,12227,23.013034
7,12227,18.066587
8,12227,20.429533
9,12227,25.518494


In [54]:
solution = solution.groupby('user.id', as_index=False)['age'].mean()

In [55]:
solution.to_csv("solutions/baselinelr.csv", index = False)